In [ ]:
# Import all needed libraries
print("Hello World")


In [ ]:
#Import all the librarys needed
import pandas as pd
import numpy as np
from scipy import stats
from numpy import mean
from numpy import std

In [ ]:
# Run previous ipynb files
%run extractData.ipynb


In [ ]:
MAIN_GRID[2][0]

In [ ]:
#load the data from repo to df
import pandas as pd
url ='https://raw.githubusercontent.com/zsaordenio/global-warming-usa/master/data/climateKnowledge/pr_1901_2016_USA.csv'
df1 = pd.read_csv(url, error_bad_lines=False)
df1.info()

In [ ]:
df1.columns

In [ ]:
#Filter data set to 1990-2015 and the columns Year and Rainfall (MM)
#start = 1990
#end = 2015
#l = list(df1_filter[' Year']) #ordered list of years
#subl = l[l.index(start):l.index(end)+1] #list of years between the start and end
#df1_filter[df1_filter[' Year'].isin(subl)] #filter dataset for list of names
df1_filter = df1.iloc[0:,[0,1]]
df1_filter = (df1_filter.set_index([' Year']).loc[1990:2015]).reset_index()
df1_filter

In [ ]:
#load the data from repo to df
import pandas as pd
url1 ='https://github.com/zsaordenio/global-warming-usa/raw/master/data/climateKnowledge/tas_1901_2016_USA.csv'
df2 = pd.read_csv(url1, error_bad_lines=False)
df2.info()

In [ ]:
df2.columns
#df2_filter.info()

In [ ]:
#Filter data set to 1990-2015 and the columns Year and Rainfall (MM)
df2_filter = df2.iloc[0:,[0,1]]
df2_filter = (df2_filter.set_index([' Year']).loc[1990:2015]).reset_index()
df2_filter

In [ ]:
#load the data from repo to df
import pandas as pd
url2 ='https://github.com/zsaordenio/global-warming-usa/raw/master/data/epa/ghg-concentrations.csv'
df3 = pd.read_csv(url2, error_bad_lines=False)
df3.info()

In [ ]:
df3

In [ ]:
df3.columns

In [ ]:
#rename column name
df3 = df3.rename(index=str, columns={"Year (negative values = BC)": "year", "Mauna Loa, Hawaii": "ml_hawai"})

In [ ]:
#Filter data set to 1990-2015 and the columns Year and Rainfall (MM)
df3_filter = df3.iloc[0:,[0,4]]
#df3_filter['Year (negative values = BC)'].astype(str).astype(int)
#df3_filter = (df3_filter.set_index(['Year (negative values = BC)']).loc[1990:2015]).reset_index()
df3_filter.info()

In [ ]:
#df3.isna().sum()
df3_subset = df3.select_dtypes(exclude=[np.number])

In [ ]:
df3_subset

In [ ]:
df3_filter_year = pd.to_numeric(df3_filter['year'], errors="coerce")

In [ ]:
df3_filter_year.dtypes

In [ ]:
df3_filter_year

In [ ]:
df3_filter_year.columns

In [ ]:
df3_filter_year.isna().sum()
#df3_filter_year.astype(int)

In [ ]:
df3_filter_year["year"] = df3_filter_year["year"].astype('Int64')

In [ ]:
# using a .concat() method
#frames = [df3_filter, df3_filter_year]
 #res1 = pd.concat(frames)
#res1

In [ ]:
df3_filter_year

In [ ]:
# summarize
from statistics import median 
print('mean=%.3f median =%.3f stdv=%.3f' % (mean(df1["Rainfall - (MM)"]), median(df1["Rainfall - (MM)"]), std(df1["Rainfall - (MM)"])))

In [ ]:
# histogram plot #Normality Check
import pandas as pd
import altair as alt
import numpy as np

alt.Chart(df1).mark_bar().encode(
    alt.X("Rainfall - (MM):Q",
          bin= True), #alt.BinParams(maxbins=100)
    y='count(*):Q'
)

In [ ]:
#Scatter Plot # Distribution
import altair as alt
from vega_datasets import data

alt.Chart(df1).mark_circle().encode(
    alt.X('Rainfall - (MM):Q', scale=alt.Scale(zero=False)),
    alt.Y('count(*):Q', scale=alt.Scale(zero=False, padding=1)),
    #color='foot_size:Q',
    #size='Rainfall - (MM):Q'
)


In [ ]:
from scipy import stats
k2, p = stats.normaltest(df1["Rainfall - (MM)"])
alpha = 1e-3
print("p = {:g}".format(p))

if p < alpha:  # null hypothesis: x comes from a normal distribution
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")
#The null hypothesis can be rejected